In [1]:
import boto3
import random
import datetime
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

pd.options.mode.chained_assignment = None

!pip install --quiet --disable-pip-version-check sns
import seaborn as sns

In [2]:
!unzip -o dataset/brazilian-ecommerce.zip -d dataset/; chmod 777 dataset/*.csv

Archive:  dataset/brazilian-ecommerce.zip
  inflating: dataset/olist_customers_dataset.csv  
  inflating: dataset/olist_sellers_dataset.csv  
  inflating: dataset/olist_order_reviews_dataset.csv  
  inflating: dataset/olist_order_payments_dataset.csv  
  inflating: dataset/product_category_name_translation.csv  
  inflating: dataset/olist_products_dataset.csv  
  inflating: dataset/olist_order_items_dataset.csv  
  inflating: dataset/olist_orders_dataset.csv  
  inflating: dataset/olist_geolocation_dataset.csv  


In [3]:
dfCustomers = pd.read_csv('dataset/olist_customers_dataset.csv')
dfProducts = pd.read_csv('dataset/olist_products_dataset.csv')
dfProductsEnglish = pd.read_csv('dataset/product_category_name_translation.csv')
dfOrders = pd.read_csv('dataset/olist_orders_dataset.csv')
dfOrdersItems = pd.read_csv('dataset/olist_order_items_dataset.csv')

In [4]:
dfCustomers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [5]:
dfProducts.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [6]:
dfOrders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [7]:
dfOrdersItems.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [8]:
dfProductsEnglish.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [9]:
names = dfProductsEnglish.set_index('product_category_name')['product_category_name_english']
dfProducts['product_category_english'] = dfProducts['product_category_name'].map(names)

Users = dfCustomers[['customer_unique_id', 'customer_zip_code_prefix', 'customer_state']].drop_duplicates().rename(
    columns={'customer_unique_id': 'USER_ID', 'customer_zip_code_prefix': 'ZipCode', 'customer_state': 'State'})

Products = dfProducts[['product_id', 'product_category_english']].drop_duplicates().rename(
    columns={'product_id':'ITEM_ID', 'product_category_english':'CATEGORY'})

Orders = dfOrders[['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp']].drop_duplicates().rename(
    columns={'order_id': 'OrderID', 'customer_id': 'USER_ID', 'order_status': 'Status', 
             'order_purchase_timestamp': 'Timestamp'})

OrderItems = dfOrdersItems[['order_id', 'product_id']].drop_duplicates().rename(
    columns={'order_id': 'OrderID', 'product_id': 'ITEM_ID'})

In [10]:
Users = Users[pd.notnull(Users['USER_ID'])]
Users = Users[pd.notnull(Users['ZipCode'])]
Users = Users[pd.notnull(Users['State'])]
Users = Users.drop_duplicates(subset='USER_ID')
reversedProducts = Users[['USER_ID', 'ZipCode', 'State']].rename(columns={'USER_ID': 'ITEM_ID'})
Products = Products[pd.notnull(Products['ITEM_ID'])]
Products = Products[pd.notnull(Products['CATEGORY'])]
Products = Products.drop_duplicates(subset='ITEM_ID')
reversedUsers = Products[['ITEM_ID', 'CATEGORY']].rename(columns={'ITEM_ID': 'USER_ID'})
Orders = Orders[pd.notnull(Orders['OrderID'])]
Orders = Orders[pd.notnull(Orders['USER_ID'])]
Orders = Orders[pd.notnull(Orders['Status'])]
Orders = Orders[pd.notnull(Orders['Timestamp'])]
Orders = Orders.drop_duplicates(subset='OrderID')
Orders['Timestamp'] = pd.to_datetime(Orders['Timestamp'], format='%Y/%m/%d %H:%M:%S')
OrderItems = OrderItems[pd.notnull(OrderItems['OrderID'])]
OrderItems = OrderItems[pd.notnull(OrderItems['ITEM_ID'])]

def toTimestamp(x): # if run twice, all values become '1'
    return(x.timestamp())

Orders['Timestamp'] = Orders.Timestamp.apply(lambda x: int(toTimestamp(x)))

mask = ['unavailable', 'canceled']
for item in mask:
    Orders = Orders[(Orders['Status']!=item)]
    
OrderItems = Orders.merge(OrderItems, how='left')

In [11]:
Users.head()

,USER_ID,ZipCode,State
0,861eff4711a542e4b93843c6dd7febb0,14409,SP
1,290c77bc529b7ac935b93aa66c333dc3,9790,SP
2,060e732b5b29e8181a18229c7b0b2b5e,1151,SP
3,259dac757896d24d7702b9acbbff3f3c,8775,SP
4,345ecd01c38d18a9036ed96c73b8d066,13056,SP


In [12]:
Products.head()

,ITEM_ID,CATEGORY
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,art
2,96bd76ec8810374ed1b65e291975717f,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,baby
4,9dc1a7de274444849c219cff195d0b71,housewares


In [13]:
OrderItems.head()

,OrderID,USER_ID,Status,Timestamp,ITEM_ID
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,1506941793,87285b34884572647811a353c7ac498a
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,1532464897,595fac2a385ac33a80bd5114aec74eb8
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,1533717529,aa4383b373c6aca5d8797843e5594415
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,1511033286,d0b61bfb1de832b15ba9d266ca96e5b0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,1518556719,65266b2da20d04dbe00c5c2d3bb7859e


In [14]:
# produtos mais vendidos -- top 100
OrderItems['ITEM_ID'].value_counts()[0:100].head(10)

99a4788cb24856965c36a24e339b6058    466
aca2eb7d00ea1a7b8ebd4e68314663af    431
422879e10f46682990de24d770e7f83d    352
d1c427060a0f73f6b889a5c7c61f2ac4    321
389d119b48cf3043d311335e499d9c6b    310
53b36df67ebb7c41585e8d54d6772e08    306
368c6c730842d78016ad823897a372db    291
53759a2ecddad2bb87a079a1f1519f73    287
154e7e31ebfa092203795c972e5804a6    269
2b4609f8948be18874494203496bc318    258
Name: ITEM_ID, dtype: int64

In [15]:
# produtos menos vendidos -- bottom 100
OrderItems['ITEM_ID'].value_counts()[1703:1803].tail(10)

57552a168008a60472e3e6bb351422e7    9
e27c4dd075a521097a085bced0631ce7    9
1d974de82e60d653a2d1cc79984dcec8    9
e07cf6c8c490ceac43740740459d87ff    9
2eb384017334e47db9ccf364216799df    9
259bd66612b61aa4e82ce3cde9272bbc    9
310b40ec41fbfc061e5309006482e68a    9
a09fb9f597a4b8a13ab0c72d70c77081    9
f22c9d6ce2431ff939aad39e026cc7ef    9
c5841303f830680eb1cdda8be0ff35bf    9
Name: ITEM_ID, dtype: int64

In [16]:
# itens por pedido
print('pedido com mais itens:', int(OrderItems['OrderID'].value_counts().max()))
print('media de itens por pedido:', int(OrderItems['OrderID'].value_counts().median()))

pedido com mais itens: 8
media de itens por pedido: 1


In [17]:
OrderItems = OrderItems[['USER_ID', 'ITEM_ID', 'Timestamp']]

Products.to_csv('products-olist.csv', sep=',', encoding='utf-8', index=False)
Users.to_csv('users-olist.csv', sep=',', encoding='utf-8', index=False)
OrderItems.to_csv('orderItems-olist.csv', sep=',', encoding='utf-8', index=False)

In [18]:
s3 = boto3.client('s3')
bucket = 'personalize-lcm'
for file in ['products-olist.csv', 'users-olist.csv', 'orderItems-olist.csv']:
    s3.upload_file(file, bucket, 'olist/' + file)